In [1]:
from app.utils.graph_utils import Neo4jConnection

In [2]:
# You can connect using either the config file or the session_state connection
# Option 1: Use the config file (works outside of the Streamlit app)
neocon = Neo4jConnection(config_file='.db_config_auto.yaml')

# Option 2: Use the session_state connection (works when run from within the Streamlit app)
# This will reuse the existing connection from the app if available
# neocon = Neo4jConnection(use_session_state=True)

# Helper function to check connection status
def check_connection(connection):
    try:
        result = connection.query_to_value("RETURN 'Connected' AS status")
        print(f"✅ Connection Status: {result}")
        return True
    except Exception as e:
        print(f"❌ Connection Error: {e}")
        print("Try connecting through the app first or check your .db_config_auto.yaml file")
        return False

# Check the connection
check_connection(neocon)

In [3]:
f_index = neocon.query_to_dataframe("""
MATCH (f)
WHERE 'Folder' IN LABELS(f)
OR 'File' IN LABELS(f)
RETURN f.filepath AS filepath, labels(f) AS labels
""")

In [ ]:
f_index = neocon.query_to_dataframe("""
MATCH (f)
WHERE 'Folder' IN LABELS(f)
OR 'File' IN LABELS(f)
RETURN f.filepath AS filepath, labels(f) AS labels
""")

In [4]:
def collapse_labels(df, col='labels'):
    return df[col].apply(lambda x: 
                         x[0] if len(x) == 1 
                         else f"MULTILABEL_{'_'.join(x)}")

collapse_labels(f_index).value_counts()

labels
File      23379
Folder     1800
Name: count, dtype: int64

In [5]:
f_index['Label'] = collapse_labels(f_index)

In [6]:
f_index.drop(columns=['labels'], inplace=True)

In [7]:
f_index

,filepath,Label
0,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,Folder
1,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,Folder
2,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,Folder
3,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,File
4,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,File
...,...,...
25174,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,File
25175,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,File
25176,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,File
25177,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,File


In [ ]:
folder

In [2]:
df['Folder'].iloc[0]

'/mnt/server/bmc-lab6/atwai/archive/ultrasound/Will Freed-Pastor/06-06-21_2021-06-06-18-00-06_Guest/20210606220006938'

In [3]:
base = "/mnt/server/bmc-lab6/atwai/archive/ultrasound/Will Freed-Pastor/"
df['Folder_Branch'] = df['Folder'].apply(lambda x: x.replace(base, ''))
df['Folder_Name'] = df['Folder_Branch'].apply(lambda x: x.rstrip('/').split('/')[-1])

In [4]:
df['Folder_Name'].value_counts()

Folder_Name
20220811180828497    2
20220811181048677    2
20220811181614333    2
20220811181813956    2
20220811182028432    2
                    ..
20220106195024950    1
20220106194747125    1
20220106194427533    1
20220106194151201    1
20220106201709562    1
Name: count, Length: 1610, dtype: int64

In [5]:
value_counts = df['Folder_Name'].apply(lambda x: x[:8]).value_counts()

In [6]:
for _v in value_counts.keys():
    if '-' in _v:
        print(_v)

8-17-22 
8-11-22 
9-23-21 
09-02-21
09-08-22
1-19-21 
06-12-21
06-06-21
09-16-21
2-18-22 
1-6-2022
1-28-22 
1-20-21 
1-21-20 
10-14-21
10-06-21
10-21-21
10-28-21
4-7-22 U
4-27-22 
2-7-22 U
4-22-22 
11-3-21 
11-11-21
5-18-22 
5-4-22 U
6-2-22 U
5-10-22 
5-25-22 
KP 1-19-
9-30-21 
US 4-21-
8-25-22 
9-19-22 


In [17]:
populated_dates = set()
verbose=True
for _v, _c in value_counts.items():
    if verbose:
        print(f"{_v} {_c}")
    if _c > 2:
        populated_dates.add(_v)

20210930 122
20220817 110
20220811 102
20200911 93
20220106 88
20220119 81
20210903 78
20210113 75
20210223 72
20200910 59
20210120 56
20210917 55
20211007 54
20210902 50
20210127 47
20210210 46
20210203 46
20210923 38
20220908 38
20220128 31
20211014 30
20220407 29
20220207 26
20211021 25
20220825 25
20220218 22
20210115 22
20220504 21
20220427 21
20210916 21
20210224 20
20220511 19
20220518 18
20211028 18
20220525 15
20220919 13
20210217 11
20220421 11
20220602 9
20210606 8
20210612 7
20211111 6
20211103 6
Optimiza 5
20220422 4
20210225 3
8-17-22  2
8-11-22  2
9-23-21  2
09-02-21 2
09-08-22 2
1_13_21  2
09_11_20 1
1-19-21  1
09_10_20 1
06-12-21 1
06-06-21 1
09-16-21 1
2-18-22  1
1-6-2022 1
1-28-22  1
1-20-21  1
1-21-20  1
10-14-21 1
10-06-21 1
10-21-21 1
10-28-21 1
4-7-22 U 1
4-27-22  1
2-7-22 U 1
4-22-22  1
11-3-21  1
11-11-21 1
5-18-22  1
5-4-22 U 1
6-2-22 U 1
5-10-22  1
5-25-22  1
KP 1-19- 1
9-30-21  1
US 4-21- 1
8-25-22  1
9-19-22  1


In [8]:
df['SessionGroup'] = df['Folder_Name'].apply(lambda x: x[:8] if x[:8] in populated_dates else 'TBD')

In [9]:
df['Files'].apply(lambda x: [xi.replace(base,'') for xi in x.strip('[').strip(']').split(',')])

0       [06-06-21_2021-06-06-18-00-06_Guest/2021060622...
1       [06-06-21_2021-06-06-18-00-06_Guest/2021060622...
2       [06-06-21_2021-06-06-18-00-06_Guest/2021060622...
3       [06-06-21_2021-06-06-18-00-06_Guest/2021060622...
4       [06-06-21_2021-06-06-18-00-06_Guest/2021060622...
                              ...                        
1794    [9-19-22 US_2022-09-19-10-14-21_Administrator/...
1795    [9-19-22 US_2022-09-19-10-14-21_Administrator/...
1796    [9-19-22 US_2022-09-19-10-14-21_Administrator/...
1797    [9-19-22 US_2022-09-19-10-14-21_Administrator/...
1798    [9-19-22 US_2022-09-19-10-14-21_Administrator/...
Name: Files, Length: 1799, dtype: object

In [10]:
df['Contents'] = df[['Files', 'Folder_Branch']].apply(lambda x: [xi.replace(base+x['Folder_Branch']+'/','') for xi in x['Files'].strip('[').strip(']').split(',')], axis=1)

In [11]:
df['LabelSession'] = 'Session'

In [12]:
df['filepath'] = df['Folder']

In [13]:
df['Label'] = 'SessionGroup'

In [14]:
df.rename(columns={'SessionGroup': 'Name'}, inplace=True)

In [15]:
df

,Folder,Files,Folder_Branch,Folder_Name,Name,Contents,LabelSession,filepath,Label
0,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,[/mnt/server/bmc-lab6/atwai/archive/ultrasound...,06-06-21_2021-06-06-18-00-06_Guest/20210606220...,20210606220006938,20210606,"[20210606220456290.vxml, 20210606220502583.vx...",Session,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,SessionGroup
1,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,[/mnt/server/bmc-lab6/atwai/archive/ultrasound...,06-06-21_2021-06-06-18-00-06_Guest/20210606220...,20210606220726806,20210606,"[20210606221114653.vxml, 20210606221120276.vx...",Session,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,SessionGroup
2,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,[/mnt/server/bmc-lab6/atwai/archive/ultrasound...,06-06-21_2021-06-06-18-00-06_Guest/20210606221...,20210606221313289,20210606,"[20210606221500802.vxml, 20210606221508335.vx...",Session,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,SessionGroup
3,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,[/mnt/server/bmc-lab6/atwai/archive/ultrasound...,06-06-21_2021-06-06-18-00-06_Guest/20210606221...,20210606221605758,20210606,"[20210606221822673.vxml, 20210606221830172.vx...",Session,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,SessionGroup
4,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,[/mnt/server/bmc-lab6/atwai/archive/ultrasound...,06-06-21_2021-06-06-18-00-06_Guest/20210606221...,20210606221941995,20210606,"[20210606222133738.vxml, 20210606222139220.vx...",Session,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,SessionGroup
...,...,...,...,...,...,...,...,...,...
1794,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,[/mnt/server/bmc-lab6/atwai/archive/ultrasound...,9-19-22 US_2022-09-19-10-14-21_Administrator/2...,20220919143550078,20220919,"[20220919143638633.vxml, 20220919143641132.vx...",Session,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,SessionGroup
1795,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,[/mnt/server/bmc-lab6/atwai/archive/ultrasound...,9-19-22 US_2022-09-19-10-14-21_Administrator/2...,20220919143914534,20220919,"[20220919144149761.vxml, 20220919144153367.vx...",Session,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,SessionGroup
1796,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,[/mnt/server/bmc-lab6/atwai/archive/ultrasound...,9-19-22 US_2022-09-19-10-14-21_Administrator/2...,20220919144251461,20220919,"[20220919144333063.vxml, 20220919144335442.vx...",Session,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,SessionGroup
1797,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,[/mnt/server/bmc-lab6/atwai/archive/ultrasound...,9-19-22 US_2022-09-19-10-14-21_Administrator/2...,20220919144416390,20220919,"[20220919144540104.vxml, 20220919144543537.vx...",Session,/mnt/server/bmc-lab6/atwai/archive/ultrasound/...,SessionGroup


In [16]:
df.to_csv('data/wfp_session_groups.csv')

In [32]:
meta = pd.read_excel('data/Metadata_Preclinical_Trial_1.xlsx', sheet_name=None)

In [34]:
meta.keys()

dict_keys(['Tumor Volume', 'Mouse Weights'])

In [36]:
meta['Tumor Volume']

,Mouse ID,Cage #,Vital Status,Trial,Unnamed: 4,Non?,2019-11-04 00:00:00,2019-11-05 00:00:00,2019-11-06 00:00:00,Unnamed: 9,...,2019-12-09 00:00:00,2019-12-16 00:00:00,2019-12-21 00:00:00,2020-01-01 00:00:00,2020-03-03 00:00:00,Unnamed: 23,Pancreas Weight (mg),Spleen Weight (mg),Necropsy Notes,Date of Death
0,KM0101,1.0,NaN,NaN,NaN,NaN,NON,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KM0102,1.0,NaN,NaN,NaN,NON,NON,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KM0103,1.0,FD (1/31/20),NaN,A,NaN,101.028,NaN,NaN,NaN,...,321.348,284.801,178.614,522.387,NaN,NaN,NaN,NaN,NaN,FD (1/31/20)
3,KM0104,1.0,FD (1/22/20),NaN,A,NaN,9.721,NaN,NaN,NaN,...,217.831,337.464,310.907,574.365,NaN,NaN,NaN,NaN,NaN,FD (1/22/20)
4,KM0105,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,MB9113,NaN,NaN,ACT,OT-I (2e6),NaN,NaN,NaN,47.642,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,465.0,228.0,mScar+ tumor,2019-11-26 00:00:00
151,MB9114,NaN,NaN,ACT,OT-I (2e6),NaN,NaN,NaN,58.875,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,368.0,247.0,mScar+ tumor,2019-11-26 00:00:00
152,MB9142,NaN,NaN,ACT-omit,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,MB9143,NaN,NaN,ACT,Control (PBS),NaN,NaN,NaN,2.545,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,219.0,113.0,"mScar+ tumor, invading liver",2019-11-26 00:00:00


In [41]:
for _k, _v in meta.items():
    print()
    print(f"SHEET: {_k}")
    print()
    for _col in _v.columns:
        print(_v[_col].value_counts())
        print()


SHEET: Tumor Volume

Mouse ID
KM0101    1
KM0102    1
KM0103    1
KM0104    1
KM0105    1
         ..
MB9113    1
MB9114    1
MB9142    1
MB9143    1
MB9196    1
Name: count, Length: 155, dtype: int64

Cage #
1.0     5
2.0     5
3.0     5
4.0     5
5.0     5
6.0     5
7.0     5
8.0     5
9.0     5
10.0    5
11.0    5
12.0    5
13.0    5
14.0    5
15.0    5
16.0    5
17.0    5
18.0    5
19.0    5
20.0    5
21.0    5
22.0    5
23.0    5
24.0    5
Name: count, dtype: int64

Vital Status
Sac (2/14/20)                       14
FD (1/13/20)                         6
FD (2/18/19)                         4
FD (1/18/20)                         4
FD (1/6/20)                          3
Sac (9/14/20)                        3
DIED (01/01/20)                      3
SAC (2/6/20)                         2
FD (1/25/20)                         2
FD (1/27/20)                         2
FD (2/3/20)                          2
SAC (1/27/20)                        2
FD (12/11/19)                        2
FD 

In [42]:
meta['Mouse Weights']

,Mouse ID,Arm,Vital Status,Unnamed: 3,2019-11-25 00:00:00,2019-12-02 00:00:00,2019-12-09 00:00:00,2019-12-16 00:00:00,2019-12-21 00:00:00,2020-01-01 00:00:00
0,KM0103,A,NaN,NaN,28,29,30,30,29,30
1,KM0104,A,NaN,NaN,22,22,23,24,24,25
2,KM0106,A,FD (12/9/19),NaN,24,25,18,FD,FD,FD
3,KM0107,A,NaN,NaN,24,26,26,26,26,26
4,KM0109,A,NaN,NaN,26,27,27,28,28,28
...,...,...,...,...,...,...,...,...,...,...
56,KM0213,B,NaN,NaN,27,25,26,27,28,28
57,KM0214,B,NaN,NaN,26,26,27,28,28,30
58,KM0216,B,NaN,NaN,25,25,25,29,25,26
59,KM0217,B,NaN,NaN,23,24,24,24,25,25


In [86]:
mouse = meta['Mouse Weights'][['Mouse ID','Arm', 'Vital Status']].copy()

mouse.rename(
    columns={
        'Mouse ID': 'sid',
        'Arm': 'arm',
        'Vital Status': 'vitalStatus'
    },
    inplace=True
)

mouse

,sid,arm,vitalStatus
0,KM0103,A,NaN
1,KM0104,A,NaN
2,KM0106,A,FD (12/9/19)
3,KM0107,A,NaN
4,KM0109,A,NaN
...,...,...,...
56,KM0213,B,NaN
57,KM0214,B,NaN
58,KM0216,B,NaN
59,KM0217,B,NaN


In [116]:
mouse['Label'] = 'Mouse'

In [118]:
mouse.to_csv('data/mouse_index.csv')

In [87]:
meta['Mouse Weights']['Unnamed: 3'].unique()

array([nan])

In [100]:
import datetime

_test = meta['Mouse Weights'].columns[4]

_test.date()

datetime.date(2019, 11, 25)

In [121]:
print(_test.date())

2019-11-25


In [122]:
weight = {}
for _col in meta['Mouse Weights'].columns:
    if isinstance(_col, datetime.datetime):
        weight[_col] = meta['Mouse Weights'][['Mouse ID', _col]].copy()
        weight[_col].rename(columns={
            'Mouse ID': 'sid',
            _col: 'weight'
        }, inplace=True)
        weight[_col]['survived'] = weight[_col]['weight'].apply(lambda x: False if 'FD' in str(x) else True)
        weight[_col]['Label'] = 'Measurement'
        weight[_col]['modality'] = 'Weight'
        weight[_col]['date'] = _col.date()

weight.keys()

dict_keys([datetime.datetime(2019, 11, 25, 0, 0), datetime.datetime(2019, 12, 2, 0, 0), datetime.datetime(2019, 12, 9, 0, 0), datetime.datetime(2019, 12, 16, 0, 0), datetime.datetime(2019, 12, 21, 0, 0), datetime.datetime(2020, 1, 1, 0, 0)])

In [123]:
for _date, _weights in weight.items():
    print(_date.date())
    print(_weights)
    _weights.to_csv(f"data/{_date.date()}_weights.csv")

2019-11-25
       sid  weight  survived        Label modality        date
0   KM0103      28      True  Measurement   Weight  2019-11-25
1   KM0104      22      True  Measurement   Weight  2019-11-25
2   KM0106      24      True  Measurement   Weight  2019-11-25
3   KM0107      24      True  Measurement   Weight  2019-11-25
4   KM0109      26      True  Measurement   Weight  2019-11-25
..     ...     ...       ...          ...      ...         ...
56  KM0213      27      True  Measurement   Weight  2019-11-25
57  KM0214      26      True  Measurement   Weight  2019-11-25
58  KM0216      25      True  Measurement   Weight  2019-11-25
59  KM0217      23      True  Measurement   Weight  2019-11-25
60  KM0220      25      True  Measurement   Weight  2019-11-25

[61 rows x 6 columns]
2019-12-02
       sid  weight  survived        Label modality        date
0   KM0103      29      True  Measurement   Weight  2019-12-02
1   KM0104      22      True  Measurement   Weight  2019-12-02
2   KM0106

In [238]:
tumor = meta['Tumor Volume'].copy()

In [239]:
tumor

,Mouse ID,Cage #,Vital Status,Trial,Unnamed: 4,Non?,2019-11-04 00:00:00,2019-11-05 00:00:00,2019-11-06 00:00:00,Unnamed: 9,...,2019-12-09 00:00:00,2019-12-16 00:00:00,2019-12-21 00:00:00,2020-01-01 00:00:00,2020-03-03 00:00:00,Unnamed: 23,Pancreas Weight (mg),Spleen Weight (mg),Necropsy Notes,Date of Death
0,KM0101,1.0,NaN,NaN,NaN,NaN,NON,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KM0102,1.0,NaN,NaN,NaN,NON,NON,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KM0103,1.0,FD (1/31/20),NaN,A,NaN,101.028,NaN,NaN,NaN,...,321.348,284.801,178.614,522.387,NaN,NaN,NaN,NaN,NaN,FD (1/31/20)
3,KM0104,1.0,FD (1/22/20),NaN,A,NaN,9.721,NaN,NaN,NaN,...,217.831,337.464,310.907,574.365,NaN,NaN,NaN,NaN,NaN,FD (1/22/20)
4,KM0105,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,MB9113,NaN,NaN,ACT,OT-I (2e6),NaN,NaN,NaN,47.642,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,465.0,228.0,mScar+ tumor,2019-11-26 00:00:00
151,MB9114,NaN,NaN,ACT,OT-I (2e6),NaN,NaN,NaN,58.875,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,368.0,247.0,mScar+ tumor,2019-11-26 00:00:00
152,MB9142,NaN,NaN,ACT-omit,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,MB9143,NaN,NaN,ACT,Control (PBS),NaN,NaN,NaN,2.545,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,219.0,113.0,"mScar+ tumor, invading liver",2019-11-26 00:00:00


In [240]:
tumor.rename(
    columns={
        'Mouse ID': 'sid',
        'Cage #': 'cageNum',
        'Vital Status': 'vitalStatus',
        'Trial': 'trial',
        'Unnamed: 4': 'trial_detail',
        'Non?': 'non'
    },
    inplace=True
)

In [241]:
tumor

,sid,cageNum,vitalStatus,trial,trial_detail,non,2019-11-04 00:00:00,2019-11-05 00:00:00,2019-11-06 00:00:00,Unnamed: 9,...,2019-12-09 00:00:00,2019-12-16 00:00:00,2019-12-21 00:00:00,2020-01-01 00:00:00,2020-03-03 00:00:00,Unnamed: 23,Pancreas Weight (mg),Spleen Weight (mg),Necropsy Notes,Date of Death
0,KM0101,1.0,NaN,NaN,NaN,NaN,NON,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KM0102,1.0,NaN,NaN,NaN,NON,NON,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KM0103,1.0,FD (1/31/20),NaN,A,NaN,101.028,NaN,NaN,NaN,...,321.348,284.801,178.614,522.387,NaN,NaN,NaN,NaN,NaN,FD (1/31/20)
3,KM0104,1.0,FD (1/22/20),NaN,A,NaN,9.721,NaN,NaN,NaN,...,217.831,337.464,310.907,574.365,NaN,NaN,NaN,NaN,NaN,FD (1/22/20)
4,KM0105,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,MB9113,NaN,NaN,ACT,OT-I (2e6),NaN,NaN,NaN,47.642,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,465.0,228.0,mScar+ tumor,2019-11-26 00:00:00
151,MB9114,NaN,NaN,ACT,OT-I (2e6),NaN,NaN,NaN,58.875,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,368.0,247.0,mScar+ tumor,2019-11-26 00:00:00
152,MB9142,NaN,NaN,ACT-omit,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,MB9143,NaN,NaN,ACT,Control (PBS),NaN,NaN,NaN,2.545,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,219.0,113.0,"mScar+ tumor, invading liver",2019-11-26 00:00:00


In [190]:
mouse2 = tumor[['sid', 'cageNum', 'vitalStatus', 'trial', 'trial_detail', 'non']].copy()
mouse2.to_csv('data/mouse_index_updated.csv')

In [243]:
tumor['Unnamed: 23'].value_counts()

Unnamed: 23
hard to see    4
2 tumors?      1
Name: count, dtype: int64

In [244]:
print("\n    Datetimes\n")
for _col in tumor.columns:
    if isinstance(_col, datetime.datetime):
        print(_col)
print()
print('\n    Other\n')
for _col in tumor.columns:
    if not isinstance(_col, datetime.datetime):
        print(_col)


    Datetimes

2019-11-04 00:00:00
2019-11-05 00:00:00
2019-11-06 00:00:00
2019-11-11 00:00:00
2019-11-12 00:00:00
2019-11-18 00:00:00
2019-11-19 00:00:00
2019-11-25 00:00:00
2019-12-02 00:00:00
2019-12-09 00:00:00
2019-12-16 00:00:00
2019-12-21 00:00:00
2020-01-01 00:00:00
2020-03-03 00:00:00


    Other

sid
cageNum
vitalStatus
trial
trial_detail
non
Unnamed: 9
Unnamed: 12
Start therapy here
Unnamed: 23
Pancreas Weight (mg)
Spleen Weight (mg)
Necropsy Notes
Date of Death


In [245]:
tumor.rename(
    columns={
        'Date of Death': 'dateOfDeath', 
        'Pancreas Weight (mg)': 'weightPancrease_mg', 
        'Spleen Weight (mg)': 'weightSpleen_mg', 
        'Necropsy Notes': 'notes'
    }, 
    inplace=True
)

In [246]:
tumor

,sid,cageNum,vitalStatus,trial,trial_detail,non,2019-11-04 00:00:00,2019-11-05 00:00:00,2019-11-06 00:00:00,Unnamed: 9,...,2019-12-09 00:00:00,2019-12-16 00:00:00,2019-12-21 00:00:00,2020-01-01 00:00:00,2020-03-03 00:00:00,Unnamed: 23,weightPancrease_mg,weightSpleen_mg,notes,dateOfDeath
0,KM0101,1.0,NaN,NaN,NaN,NaN,NON,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KM0102,1.0,NaN,NaN,NaN,NON,NON,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KM0103,1.0,FD (1/31/20),NaN,A,NaN,101.028,NaN,NaN,NaN,...,321.348,284.801,178.614,522.387,NaN,NaN,NaN,NaN,NaN,FD (1/31/20)
3,KM0104,1.0,FD (1/22/20),NaN,A,NaN,9.721,NaN,NaN,NaN,...,217.831,337.464,310.907,574.365,NaN,NaN,NaN,NaN,NaN,FD (1/22/20)
4,KM0105,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,MB9113,NaN,NaN,ACT,OT-I (2e6),NaN,NaN,NaN,47.642,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,465.0,228.0,mScar+ tumor,2019-11-26 00:00:00
151,MB9114,NaN,NaN,ACT,OT-I (2e6),NaN,NaN,NaN,58.875,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,368.0,247.0,mScar+ tumor,2019-11-26 00:00:00
152,MB9142,NaN,NaN,ACT-omit,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,MB9143,NaN,NaN,ACT,Control (PBS),NaN,NaN,NaN,2.545,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,219.0,113.0,"mScar+ tumor, invading liver",2019-11-26 00:00:00


In [247]:
necrop = tumor[['sid', 'vitalStatus', 'dateOfDeath', 'weightPancrease_mg', 'weightSpleen_mg', 'notes']]

In [248]:
necrop['sid'].unique()

array(['KM0101', 'KM0102', 'KM0103', 'KM0104', 'KM0105', 'KM0106',
       'KM0107', 'KM0108', 'KM0109', 'KM0110', 'KM0111', 'KM0112',
       'KM0113', 'KM0114', 'KM0115', 'KM0116', 'KM0117 (KM0221)',
       'KM0118', 'KM0119', 'KM0120', 'KM0121', 'KM0122', 'KM0123',
       'KM0124', 'KM0125', 'KM0126', 'KM0127', 'KM0128', 'KM0129',
       'KM0130', 'KM0131', 'KM0132', 'KM0133', 'KM0134', 'KM0135',
       'KM0136', 'KM0137', 'KM0138', 'KM0139', 'KM0140', 'KM0141',
       'KM0142', 'KM0143', 'KM0144', 'KM0145', 'KM0146', 'KM0147',
       'KM0148', 'KM0149', 'KM0150', 'KM0151', 'KM0152', 'KM0153',
       'KM0154', 'KM0155', 'KM0156', 'KM0157', 'KM0158', 'KM0159',
       'KM0160', 'KM0161', 'KM0162', 'KM0163', 'KM0164', 'KM0165',
       'KM0166 (KM0223)', 'KM0167', 'KM0168', 'KM0169', 'KM0170',
       'KM0171', 'KM0172', 'KM0173', 'KM0174', 'KM0175', 'KM0176',
       'KM0177', 'KM0178', 'KM0179', 'KM0180', 'KM0181', 'KM0182',
       'KM0183', 'KM0184', 'KM0185', 'KM0186', 'KM0187', 'KM0188

In [249]:
necrop = necrop[~(necrop['vitalStatus'].isna() & necrop['dateOfDeath'].isna())].reset_index(drop=True)
necrop['Label'] = 'Necropsy'

In [250]:
necrop

,sid,vitalStatus,dateOfDeath,weightPancrease_mg,weightSpleen_mg,notes,Label
0,KM0103,FD (1/31/20),FD (1/31/20),NaN,NaN,NaN,Necropsy
1,KM0104,FD (1/22/20),FD (1/22/20),NaN,NaN,NaN,Necropsy
2,KM0106,Sac (12/9/19),12/09/19 (sac),NaN,NaN,Panc tumor,Necropsy
3,KM0107,FD (1/2/20),FD 1/2/19,NaN,NaN,"panc tumor, kidney mets",Necropsy
4,KM0109,DIED (01/01/20),DIED (01/01/20),NaN,NaN,"Panc tumor, liver mets, no lung mets",Necropsy
...,...,...,...,...,...,...,...
98,MB9065,NaN,2019-11-26 00:00:00,217.0,151.0,mScar+ tumor,Necropsy
99,MB9113,NaN,2019-11-26 00:00:00,465.0,228.0,mScar+ tumor,Necropsy
100,MB9114,NaN,2019-11-26 00:00:00,368.0,247.0,mScar+ tumor,Necropsy
101,MB9143,NaN,2019-11-26 00:00:00,219.0,113.0,"mScar+ tumor, invading liver",Necropsy


In [251]:
mismatch = necrop[~necrop.apply(lambda x: x['vitalStatus'] == x['dateOfDeath'], axis=1)].copy().reset_index(drop=True)

In [252]:
mismatch

,sid,vitalStatus,dateOfDeath,weightPancrease_mg,weightSpleen_mg,notes,Label
0,KM0106,Sac (12/9/19),12/09/19 (sac),NaN,NaN,Panc tumor,Necropsy
1,KM0107,FD (1/2/20),FD 1/2/19,NaN,NaN,"panc tumor, kidney mets",Necropsy
2,KM0120,Sac (9/14/20),2020-09-14 00:00:00,NaN,NaN,No tumor,Necropsy
3,KM0124,FD (2/25/20),NaN,NaN,NaN,NaN,Necropsy
4,KM0137,NaN,2019-11-27 00:00:00,NaN,NaN,NA (fd),Necropsy
5,KM0140,FD (3/2/20),NaN,NaN,NaN,NaN,Necropsy
6,KM0152,NaN,FD (12/16/19),NaN,NaN,"panc. tumor, peritoneal mass",Necropsy
7,KM0157,NaN,FD (12/9/19),NaN,NaN,"Panc tumor, Lesion on spleen",Necropsy
8,KM0166 (KM0223),NaN,FD (12/9/19),NaN,NaN,"Panc tumor invading stomach, liver with mets, ...",Necropsy
9,KM0173,FD (2/18/19),NaN,NaN,NaN,NaN,Necropsy


In [253]:
necrop.to_csv('data/necropsy_index.csv')

In [192]:
tumor.columns

Index([               'sid',            'cageNum',        'vitalStatus',
                    'trial',       'trial_detail',                'non',
        2019-11-04 00:00:00,  2019-11-05 00:00:00,  2019-11-06 00:00:00,
               'Unnamed: 9',  2019-11-11 00:00:00,  2019-11-12 00:00:00,
              'Unnamed: 12',  2019-11-18 00:00:00,  2019-11-19 00:00:00,
       'Start therapy here',  2019-11-25 00:00:00,  2019-12-02 00:00:00,
        2019-12-09 00:00:00,  2019-12-16 00:00:00,  2019-12-21 00:00:00,
        2020-01-01 00:00:00,  2020-03-03 00:00:00,        'Unnamed: 23',
       'weightPancrease_mg',    'weightSpleen_mg',              'notes',
              'dateOfDeath'],
      dtype='object')

In [205]:
tumor_meas_cols = ['sid']  
for _col in tumor.columns:
    if isinstance(_col, datetime.datetime):
        tumor_meas_cols.append(_col)

col_map = {}
for _col in tumor_meas_cols:
    if isinstance(_col, datetime.datetime): 
        col_map[_col] = _col.date()

tumorMeasurements = tumor[tumor_meas_cols].rename(columns=col_map)

In [206]:
tumorMeasurements

,sid,2019-11-04,2019-11-05,2019-11-06,2019-11-11,2019-11-12,2019-11-18,2019-11-19,2019-11-25,2019-12-02,2019-12-09,2019-12-16,2019-12-21,2020-01-01,2020-03-03
0,KM0101,NON,NaN,NaN,NON,NaN,5.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KM0102,NON,NaN,NaN,NON,NaN,NON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KM0103,101.028,NaN,NaN,107.117,NaN,186.187,NaN,214.040,182.824,321.348,284.801,178.614,522.387,NaN
3,KM0104,9.721,NaN,NaN,19.751,NaN,49.931,NaN,119.882,191.924,217.831,337.464,310.907,574.365,NaN
4,KM0105,NaN,NaN,NaN,NaN,NaN,OMIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,MB9113,NaN,NaN,47.642,112.143,NaN,NaN,216.684,214.200,NaN,NaN,NaN,NaN,NaN,NaN
151,MB9114,NaN,NaN,58.875,150.531,NaN,NaN,184.366,254.267,NaN,NaN,NaN,NaN,NaN,NaN
152,MB9142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,MB9143,NaN,NaN,2.545,5.16,NaN,NaN,51.291,89.314,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:

for _col in tumorMeasurements.columns: 
    if _col != 'sid':


SyntaxError: incomplete input (1331781787.py, line 1)

In [220]:
tumorMeas = {}
for _col in tumorMeasurements.columns:
    if isinstance(_col, datetime.date):
        tumorMeas[_col] = tumorMeasurements[['sid', _col]].copy()
        tumorMeas[_col].rename(columns={
            _col: 'tumorSize'
        }, inplace=True)
        tumorMeas[_col]['noTumor'] = tumorMeas[_col]['tumorSize'].apply(lambda x: True if 'NON' in str(x) else False)
        tumorMeas[_col] = tumorMeas[_col][~tumorMeas[_col]['tumorSize'].isna()]
        tumorMeas[_col]['date'] = _col
        tumorMeas[_col]['Label'] = 'TumorVolume'

tumorMeas

{datetime.date(2019, 11, 4):                 sid tumorSize  noTumor        date        Label
 0            KM0101       NON     True  2019-11-04  TumorVolume
 1            KM0102       NON     True  2019-11-04  TumorVolume
 2            KM0103   101.028    False  2019-11-04  TumorVolume
 3            KM0104     9.721    False  2019-11-04  TumorVolume
 5            KM0106    18.144    False  2019-11-04  TumorVolume
 6            KM0107    33.639    False  2019-11-04  TumorVolume
 8            KM0109    11.254    False  2019-11-04  TumorVolume
 10           KM0111     0.316    False  2019-11-04  TumorVolume
 11           KM0112      2.67    False  2019-11-04  TumorVolume
 12           KM0113       NON     True  2019-11-04  TumorVolume
 13           KM0114     8.376    False  2019-11-04  TumorVolume
 14           KM0115    19.254    False  2019-11-04  TumorVolume
 16  KM0117 (KM0221)    10.984    False  2019-11-04  TumorVolume
 19           KM0120       NON     True  2019-11-04  TumorVolu

In [221]:
for _date, _tm in tumorMeas.items():
    print(_date)
    print(_tm)
    _tm.to_csv(f"data/{_date}_tumorMeasurement.csv")

2019-11-04
                sid tumorSize  noTumor        date        Label
0            KM0101       NON     True  2019-11-04  TumorVolume
1            KM0102       NON     True  2019-11-04  TumorVolume
2            KM0103   101.028    False  2019-11-04  TumorVolume
3            KM0104     9.721    False  2019-11-04  TumorVolume
5            KM0106    18.144    False  2019-11-04  TumorVolume
6            KM0107    33.639    False  2019-11-04  TumorVolume
8            KM0109    11.254    False  2019-11-04  TumorVolume
10           KM0111     0.316    False  2019-11-04  TumorVolume
11           KM0112      2.67    False  2019-11-04  TumorVolume
12           KM0113       NON     True  2019-11-04  TumorVolume
13           KM0114     8.376    False  2019-11-04  TumorVolume
14           KM0115    19.254    False  2019-11-04  TumorVolume
16  KM0117 (KM0221)    10.984    False  2019-11-04  TumorVolume
19           KM0120       NON     True  2019-11-04  TumorVolume
22           KM0123    56.392